<a href="https://colab.research.google.com/github/tipiorgup/Tutorial_IDP_SAPs/blob/main/Simulation%20code/Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
import os
import sys
import numpy as np
import shutil
try:
    if os.path.exists('Tutorial_IDP_SAPs'):
        print("Found existing directory, attempting to remove...")
        shutil.rmtree('Tutorial_IDP_SAPs', ignore_errors=True)
        print("Directory removed successfully")
    else:
        print("Directory doesn't exist")
except Exception as e:
    print(f"Error while trying to remove directory: {e}")

Directory doesn't exist


In [40]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
!pip install qmllib

In [41]:
!git clone https://github.com/tipiorgup/Tutorial_IDP_SAPs.git
!pip install openmm
!pip install MDAnalysis
!pip install mdtraj
!pip install nglview
!pip install ipywidgets==7.7.1 jupyter_client==6.1.12 notebook==6.5.5
import ipywidgets as widgets
from IPython.display import display
import nglview
import mdtraj

Cloning into 'Tutorial_IDP_SAPs'...
remote: Enumerating objects: 135, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 135 (delta 45), reused 93 (delta 24), pack-reused 0 (from 0)
Receiving objects: 100% (135/135), 91.93 KiB | 2.96 MiB/s, done.
Resolving deltas: 100% (45/45), done.
  Using cached ipywidgets-7.7.1-py2.py3-none-any.whl.metadata (1.9 kB)
  Using cached jupyter_client-6.1.12-py3-none-any.whl.metadata (4.3 kB)
  Using cached notebook-6.5.5-py3-none-any.whl.metadata (2.5 kB)
  Using cached widgetsnbextension-3.6.10-py2.py3-none-any.whl.metadata (1.3 kB)
INFO: pip is looking at multiple versions of jupyter-server to determine which version is compatible with other requirements. This could take a while.
  Using cached jupyter_server-2.14.2-py3-none-any.whl.metadata (8.4 kB)
  Using cached jupyter_server-2.14.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached jupyter_server-2.14.0-py3-none-any.whl.metadata (

In [42]:
os.chdir('Tutorial_IDP_SAPs/Simulation code')

In [43]:
global stored_values
stored_values = {
    'Simulation type': None,
    'Sequence': None,
    'Sequence 1': None,
    'Sequence 2': None,
    'Ratio': None,
    'Processed_Ratio': None,
    'peptide_pairs': None
}

# Create the checkbox widgets
checkbox_a = widgets.Checkbox(
    value=False,
    description='Single peptide',
    disabled=False
)

checkbox_b = widgets.Checkbox(
    value=False,
    description='Co-assembly',
    disabled=False
)

# Create text input for option A
text_input_a = widgets.Text(
    description='Sequence:',
    disabled=False,
    layout={'visibility': 'hidden', 'width': '800px'},
    placeholder='Enter sequence (up to 100 characters)'
)

# Create text inputs for option B
text_input_b1 = widgets.Text(
    description='Sequence 1:',
    disabled=False,
    layout={'visibility': 'hidden', 'width': '800px'},
    placeholder='Enter sequence 1 (up to 100 characters)'
)

text_input_b2 = widgets.Text(
    description='Sequence 2:',
    disabled=False,
    layout={'visibility': 'hidden', 'width': '800px'},
    placeholder='Enter sequence 2 (up to 100 characters)'
)

text_input_b3 = widgets.Text(
    description='Ratio:',
    disabled=False,
    layout={'visibility': 'hidden', 'width': '800px'},
    placeholder='Enter ratio'
)

def process_ratio(ratio_str):
    try:
        numbers = ratio_str.replace(' ', '').split('-')
        if len(numbers) == 2:
            num1 = int(numbers[0]) * 100
            num2 = int(numbers[1]) * 100
            return f"{num1}-{num2}"
    except:
        return ratio_str
    return ratio_str

def update_stored_values():
    if checkbox_a.value:
        stored_values['Simulation type'] = 'Single peptide'
        stored_values['Sequence'] = text_input_a.value
        stored_values['Sequence 1'] = text_input_a.value
        stored_values['Sequence 2'] = 'A'  # Constant A as Sequence 2
        stored_values['Ratio'] = '10-0'  # Default ratio
        stored_values['Processed_Ratio'] = process_ratio('10-0')  # Process default ratio
        if text_input_a.value:  # Only create if sequence exists
            stored_values['peptide_pairs'] = f"{text_input_a.value}-A"
    elif checkbox_b.value:
        stored_values['Simulation type'] = 'Co-assembly'
        stored_values['Sequence'] = None
        stored_values['Sequence 1'] = text_input_b1.value
        stored_values['Sequence 2'] = text_input_b2.value
        stored_values['Ratio'] = text_input_b3.value
        stored_values['Processed_Ratio'] = process_ratio(text_input_b3.value)
        if text_input_b1.value and text_input_b2.value:
            stored_values['peptide_pairs'] = f"{text_input_b1.value}-{text_input_b2.value}"

def on_checkbox_change(change):
    if change['owner'] == checkbox_a and change['new']:
        checkbox_b.value = False
        text_input_a.layout.visibility = 'visible'
        text_input_b1.layout.visibility = 'hidden'
        text_input_b2.layout.visibility = 'hidden'
        text_input_b3.layout.visibility = 'hidden'
    elif change['owner'] == checkbox_b and change['new']:
        checkbox_a.value = False
        text_input_a.layout.visibility = 'hidden'
        text_input_b1.layout.visibility = 'visible'
        text_input_b2.layout.visibility = 'visible'
        text_input_b3.layout.visibility = 'visible'

    update_stored_values()

def on_text_change(change):
    update_stored_values()

# Register callbacks
checkbox_a.observe(on_checkbox_change, names='value')
checkbox_b.observe(on_checkbox_change, names='value')
text_input_a.observe(on_text_change, names='value')
text_input_b1.observe(on_text_change, names='value')
text_input_b2.observe(on_text_change, names='value')
text_input_b3.observe(on_text_change, names='value')



In [44]:
widget = widgets.VBox([
    checkbox_a,
    checkbox_b,
    text_input_a,
    text_input_b1,
    text_input_b2,
    text_input_b3
])

In [45]:
display(widget)

In [46]:
stored_values

{'Simulation type': None,
 'Sequence': None,
 'Sequence 1': None,
 'Sequence 2': None,
 'Ratio': None,
 'Processed_Ratio': None,
 'peptide_pairs': None}

In [47]:
#Check in src the file parsers.py, it will show all options available
ionic_strength='0.3'

In [48]:
# First check file
residues_file = 'src/residues.csv'

if not os.path.exists(residues_file):
    print(f"❌ Error: File '{residues_file}' not found!")
else:
    !python clean_pair_code.py --ratio {stored_values['Ratio']} \
        --ionic-strength {ionic_strength} \
        --ph 7.4 \
        --peptide-pair {stored_values['peptide_pairs']} \
        --run-all \
        --residues-file 'src/residues.csv' \
        --steps 1000

Cannot import pdbfixer. This may affect 3SPN2 model.
Processing pair: None
Error processing pair None: invalid literal for int() with base 10: 'None'
Error processing pair None: invalid literal for int() with base 10: 'None'


In [73]:
name = stored_values['peptide_pairs']
# name='HHHWWW-A'

traj = mdtraj.load_dcd(f'{name}/310/{name}_slab.dcd', top=f'{name}/start.pdb')
traj.xyz -= np.mean(traj.xyz, axis=1, keepdims=True)  # realign to the origin
view = nglview.show_mdtraj(traj)

OSError: No such file: HHHWWW-A/start.pdb

In [67]:
#Get frames to obtain SLATM representation
import glob
def process_trajectory(path, sequence, ens_file):
    txt_files = glob.glob(os.path.join(path, '310', '*dcd'), recursive=True)
    if not txt_files:
        print(f"No DCD files found in {path}")
        return None

    topology = os.path.join(path, 'start.pdb')
    try:
        trajectory = md.load(txt_files[0], top=topology)

        if os.path.isfile(ens_file):
            print('The ensemble has been done')
            return None

        traj_r = trajectory[-50:]
        traj = {sequence: [traj_r[j] for j in range(50)]}

        st = np.arange(5, 50, 10)
        e = np.arange(10, 60, 10)
        ensembles = {sequence: {}}

        for idx in range(len(st)):
            ensembles[sequence][idx] = traj[sequence][st[idx]:e[idx]]

        with open(ens_file, 'wb') as handle:
            pickle.dump(ensembles, handle, protocol=pickle.HIGHEST_PROTOCOL)

        return ensembles
    except Exception as e:
        print(f"Error processing trajectory: {e}")
        return None

In [69]:
ens_file=f'{name}_ensemble.pkl'
ensembles = process_trajectory(f'{name}', name, ens_file)
if os.path.isfile(ens_file) and not ensembles:
    with open(ens_file, 'rb') as f:
        ensembles = pickle.load(f)

No DCD files found in HHHWWW-A


In [ ]:
d = {'A':1, 'R':2, 'N':3, 'D':4, 'C':5, 'Q':6, 'G':7, 'E':8, 'H':9, 'I':10,
         'L':11, 'K':12, 'M':13, 'F':14, 'P':15, 'S':16, 'T':17, 'W':18, 'Y':19, 'V':20}
if stored_values['Simulation type'] == 'Single peptide':
    try:
        error_mol=[]
        slatm={}
        numbers_n={}
        n_residues=len(stored_values['Sequence'])
        res_simple=list(stored_values['Sequence'])
        n_chains=int(1000/n_residues)
        chains=np.reshape(np.arange(ensembles[stored_values['peptide_pairs']][0][0].n_residues),(n_chains,len(stored_values['Sequence'])))
        beads=res_simple*n_chains

        numbers = {stored_values['Sequence']: [d[ni] for ni in beads]}
        slatm = {stored_values['Sequence']: {}}
        #Check original code and we create a SLATM per snapshot, here we use the last snapshot (4) of the last ensmeble (4), otherwise go into two loops
        slatm[] = []
        slatm_data = generate_slatm(
            ensembles[stored_values['peptide_pairs']][4][4].xyz,
            numbers[stored_values['Sequence']],
            mbtypes=mbtypes,
            rcut=20,
            dgrids=[1, 0.5],
            sigmas=[0.3, 0.2]
        )
        slatm[stored_values['Sequence']].append(slatm_data)

        with open(slatm_file, 'wb') as handle:
            pickle.dump(slatm, handle, protocol=pickle.HIGHEST_PROTOCOL)

        return slatm
    except Exception as e:
        print(f"Error calculating SLATM: {e}")
        return None
else:
          n_residues = len(stored_values['Sequence 1'])
          res_simple0 = list(stored_values['Sequence 1'])
          res_simple1 = list(stored_values['Sequence 2'])
          n_mol = ensembles[stored_values['peptide_pairs']][0][0].n_residues
          n_chains = int(1000/n_residues)

          chains = np.reshape(np.arange(n_mol), (n_chains, n_residues))
          beads0 = res_simple0 * (int(int(ratio)/n_residues))
          beads1 = res_simple1 * (int((1000-int(ratio))/n_residues))

          numbers = {stored_values['peptide_pairs']: [d[ni] for ni in beads0] + [d[ni] for ni in beads1]}
          slatm = {stored_values['peptide_pairs']: {}}
          slatm[stored_values['peptide_pairs']] = []
          slatm_data = generate_slatm(
              ensembles[stored_values['peptide_pairs']][j][u].xyz,
              numbers[stored_values['peptide_pairs']],
              mbtypes=mbtypes,
              rcut=20,
              dgrids=[1, 0.5],
              sigmas=[0.3, 0.2]
          )
          slatm[stored_values['peptide_pairs']].append(slatm_data)

          with open(slatm_file, 'wb') as handle:
              pickle.dump(slatm, handle, protocol=pickle.HIGHEST_PROTOCOL)

          return slatm
      except Exception as e:
          print(f"Error calculating SLATM: {e}")
          return None
